# Extraindo dados de vagas do Linkedin através de Web Scraping

### Instalando as bibliotecas necessárias

In [1]:
!pip install requests # Usada para fazer requisições HTTP, ou seja, acessar páginas da web e pegar seu conteúdo.
!pip install beautifulsoup4 # Usada para processar e interpretar o código HTML das páginas da web.


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


### Importando as Bibliotecas

In [2]:
# Importando as bibliotecas necessárias

import requests  # Para acessar páginas da web
from bs4 import BeautifulSoup  # Para extrair informações do código HTML das páginas
import pandas as pd  # Para criar e manipular tabelas de dados

### Coletando informações de filtros para a Vaga

In [ ]:
# Recebendo o nome da vaga

nome_vaga = input("Digite o nome da vaga: ")
nome_vaga_tratada = nome_vaga.replace(" ", "%20").lower()

In [ ]:
# Recebendo a experiência da vaga

experiencia = input("Digite o tipo de experiência: (0 = Todos, 1 = Estágio, 2 = Assistente, 3 = Júnior, 4 = Pleno-Sênior e 5 = Diretor)")
if experiencia == "0" or int(experiencia) > 5 or experiencia == "":
    experiencia_tratada = ''
else:
    experiencia_tratada = f'''&f_E={experiencia}'''

In [ ]:
# Recebendo o tipo da vaga

tipo_vaga = input("Digite o tipo da vaga: (0 = Todos, 1 = Presencial, 2 = Remoto, 3 = Híbrido)")
if tipo_vaga == "0" or int(tipo_vaga) > 3 or tipo_vaga == "":
    tipo_vaga_tratada = ''
else:
    tipo_vaga_tratada = f'''&f_WT={tipo_vaga}'''

### Criando a URL de busca

In [ ]:
# Usando as variáveis coletadas para criar a URL de pesquisa

base_url = f'''
https://www.linkedin.com/jobs/search?
keywords={nome_vaga_tratada}
&location=Brazil
&geoId=106057199
{experiencia_tratada}
&f_TPR=r86400
{tipo_vaga_tratada}
&position=1
&pageNum=0 
'''

In [ ]:
# Fazendo um tratamento para a URL 

base_url = base_url.replace("\n", "")

### Fazendo a requisição para o Linkedin

In [ ]:
# Faz uma requisição HTTP GET para a URL base e armazena a resposta na variável 'response'
response = requests.get(base_url)

In [11]:
print(response)

<Response [200]>


### Extraindo os dados da Página

In [ ]:
# Analisa o conteúdo HTML da resposta e cria um objeto BeautifulSoup para facilitar a extração de dados
site = BeautifulSoup(response.text, 'html.parser')


### Encontrando os elementos com informações das vagas

In [ ]:
# Encontra todas as divs que possuem a classe específica usada para identificar os cartões de vagas no site
dados = site.find_all("div", attrs={"class": "base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card"})


### Criando uma lista para armazenar os dados

In [ ]:
# Listas para armazenar os dados extraídos do site
armazenando_vaga = []            # Armazena os títulos das vagas
armazenamento_empresa = []        # Armazena os nomes das empresas
armazenamento_localizacao = []    # Armazena as localizações das vagas
armazenamento_tempo = []          # Armazena o tipo de contratação ou período (ex: tempo integral, meio período)
armazenamento_data = []           # Armazena a data de publicação das vagas
armazenamento_link_empresa = []   # Armazena os links para as páginas das empresas
armazenamento_link_vaga = []      # Armazena os links diretos para as vagas


### Extraindo informações de cada Vaga

In [ ]:
# Percorre cada elemento encontrado na busca dos cartões de vaga
for palavra in dados:
    # Nome da vaga
    nome_vaga = palavra.find("h3", attrs={"class": "base-search-card__title"})
    nome_vaga = nome_vaga.text.strip() if nome_vaga else "Sem Informação"  # Remove espaços extras e trata casos vazios
    armazenando_vaga.append(nome_vaga)  # Adiciona o nome da vaga à lista correspondente

    # Nome da empresa
    nome_empresa = palavra.find("h4", attrs={"class": "base-search-card__subtitle"})
    nome_empresa = nome_empresa.text.strip() if nome_empresa else "Sem Informação"
    armazenamento_empresa.append(nome_empresa)

    # Localização da vaga
    localizacao = palavra.find("span", attrs={"class": "job-search-card__location"})
    localizacao = localizacao.text.strip() if localizacao else "Sem Informação"
    armazenamento_localizacao.append(localizacao)

    # Tempo/disponibilidade da vaga (ex: tempo integral, meio período)
    tempo_vaga = palavra.find_all("time")[0].text.strip()
    armazenamento_tempo.append(tempo_vaga)

    # Data da publicação da vaga
    data = palavra.find_all("time")[0]
    data_trat = data["datetime"]  # Obtém a data formatada diretamente do atributo 'datetime'
    armazenamento_data.append(data_trat)

    # Link direto para a vaga
    link_vaga = palavra.find("a", attrs={"class": "base-card__full-link"})
    link_vaga = link_vaga.get("href") if link_vaga else "Sem Link"
    armazenamento_link_vaga.append(link_vaga)

    # Link da página da empresa
    link_empresa = palavra.find("a", attrs={"class": "hidden-nested-link"})
    link_empresa = link_empresa.get("href") if link_empresa else "Sem Link"  # Corrigido para verificar a variável correta
    armazenamento_link_empresa.append(link_empresa)

# Exibe a quantidade total de vagas armazenadas
print(f"Total de vagas armazenadas: {len(armazenando_vaga)}")


Total de vagas armazenadas: 49


### Criando um dicionário com os dados coletados


In [ ]:
# Criação de um dicionário para armazenar os dados coletados das vagas
base_vagas = {
    "vaga": armazenando_vaga,               # Lista com os títulos das vagas
    "empresa": armazenamento_empresa,       # Lista com os nomes das empresas
    "localizacao": armazenamento_localizacao,  # Lista com as localizações das vagas
    "tempo_postada": armazenamento_tempo,   # Lista com o tempo desde a publicação da vaga
    "data_vaga": armazenamento_data,        # Lista com a data exata de publicação da vaga
    "link_vaga": armazenamento_link_vaga,   # Lista com os links diretos para as vagas
    "link_empresa": armazenamento_link_empresa  # Lista com os links para as páginas das empresas
}


### Criando um dataframe para armazenar os dados

In [ ]:
# Converte o dicionário 'base_vagas' em um DataFrame do Pandas para facilitar a análise e manipulação dos dados
df = pd.DataFrame(base_vagas)


### Proximos passos, Fazer a inclusão dos daos em um banco de dados e subir para o Supabase

In [20]:
# Instalando o Supabase
!pip install supabase


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Importando as bibliotecas necessárias

from supabase import Client, create_client  # Biblioteca para interagir com o banco de dados Supabase
from datetime import datetime  # Biblioteca para trabalhar com datas e horários


In [ ]:
import os

supabase_url = os.getenv("SUPABASE_URL")
supabase_api_key = os.getenv("SUPABASE_API_KEY")


In [ ]:
# Criando um cliente que se conecta ao Supabase com a URL e a API Key fornecidas,
# e concedendo ao cliente permissões para fazer alterações no banco de dados
supabase: Client = create_client(supabase_url, supabase_api_key)


In [ ]:
# Adicionando uma nova coluna 'data_insercao' ao DataFrame, 
# com a data e hora atual formatada como 'YYYY-MM-DD' (ano-mês-dia)
df["data_insercao"] = datetime.now().strftime("%Y-%m-%d")


In [ ]:
# Convertendo o DataFrame 'df' para um dicionário, onde cada linha é representada como um dicionário,
# com as colunas do DataFrame sendo as chaves e os valores das colunas sendo os valores correspondentes.
# A opção 'orient="records"' cria uma lista de dicionários, onde cada dicionário corresponde a uma linha.
dados_vagas = df.to_dict(orient="records")


In [ ]:
try:
    # Tentando inserir os dados na tabela 'vagas' do Supabase
    # O método 'upsert' tenta inserir os dados, mas se encontrar um conflito na chave primária (a coluna 'link_vaga'),
    # ele irá atualizar os dados existentes ao invés de criar uma nova linha.
    response_supabase = supabase.table("vagas").upsert(dados_vagas, on_conflict="link_vaga").execute()

    # Verificando se os dados foram inseridos com sucesso
    if response_supabase.data:
        # Se a resposta contiver dados (vagas inseridas com sucesso)
        print(f"{len(dados_vagas)} vagas foram inseridas com sucesso !!!")
    else:
        # Caso contrário, imprime o erro
        print(f"Erro ao tentar inserir a vaga: {response_supabase}")
except Exception as e:
    # Captura qualquer erro que ocorra durante a operação e imprime uma mensagem de erro
    print("Erro ao tentar conectar com o Supabase!!!", str(e))


49 vagas foram inseridas com sucesso !!!


OS códigos abaixo foram feitos para estudo, criando um database local para testes. 

In [28]:
# import sqlite3
# import pandas as pd
# from datetime import datetime

# # Criar a conexao com meu banco de dados SQLITE
# nome_banco = "vagas_linkedin2.db"
# conector = sqlite3.connect(nome_banco)
# cursor = conector.cursor()

# # criando nossa tabela para armazenar as vagas

# cursor.execute('''
# CREATE TABLE IF NOT EXISTS vagas(
# id INTEGER PRIMARY KEY AUTOINCREMENT,
# vaga TEXT NOT NULL,
# empresa TEXT NOT NULL,
# localizacao TEXT NOT NULL,
# Tempo_postada TEXT NOT NULL,
# data_vaga DATE NOT NULL,
# link_vaga TEXT UNIQUE NOT NULL,
# link_empresa TEXT NOT NULL,
# data_insercao DATE NOT NULL
# )
# ''')

# # Adicionando data no Python
# data_hoje = datetime.now().strftime("%Y-%m-%d")
# df["data_insercao"] = data_hoje

# dados_linkedin = list(df.itertuples(index = False, name = None))

# cursor.executemany('''
#   INSERT OR IGNORE INTO vagas (vaga, empresa, localizacao, Tempo_postada, data_vaga, link_vaga, link_empresa, data_insercao)
#   VALUES (?, ?, ?, ?, ?, ?, ?, ?)
# ''', dados_linkedin)

# # salvar as infos no banco
# conector.commit()

# # fechar conexao
# conector.close()